In [1]:
import sys

In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...
remote: Enumerating objects: 4015, done.
remote: Counting objects: 100% (569/569), done.9)
remote: Compressing objects: 100% (205/205), done.
remote: Total 4015 (delta 340), reused 542 (delta 332), pack-reused 3446 (from 1)
Receiving objects: 100% (4015/4015), 49.69 MiB | 8.63 MiB/s, done.
Resolving deltas: 100% (2134/2134), done.


In [3]:
sys.path.insert(0,'boptestGymService')

In [6]:
import gymnasium as gym
import pandas as pd
import matplotlib.pyplot as plt
import torch

from stable_baselines3 import PPO

from stable_baselines3.common.vec_env import SubprocVecEnv

from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback
from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper

In [7]:
device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')

In [8]:
# for reproducibility
seed = 1337

In [9]:
url = 'https://api.boptest.net'

In [ ]:
# Define a function to create environments
def make_env():
    def _init():
        env = BoptestGymEnv(
            url=url,
            testcase='bestest_hydronic_heat_pump',
            actions=['oveHeaPumY_u'],
            observations={
                'time': (0, 604800),
                'reaTZon_y': (280., 310.),
                'PriceElectricPowerHighlyDynamic': (-0.4, 0.4),
                'LowerSetp[1]': (280., 310.),
                'UpperSetp[1]': (280., 310.)
            },
            predictive_period=24*3600,
            regressive_period=6*3600,
            random_start_time=True,
            max_episode_length=24*3600,
            warmup_period=24*3600,
            step_period=3600
        )
        # Normalize observations
        env = NormalizedObservationWrapper(env)
        return env
    return _init

# Number of parallel environments
num_envs = 2  # Adjust based on your system's capabilities

# Create the vectorized environment
env = SubprocVecEnv([make_env() for _ in range(num_envs)])

# Initialize the RL agent using PPO
model = PPO('MlpPolicy', env, verbose=1, device=device)

# Train the model
model.learn(total_timesteps=1)  # Increase total_timesteps as needed

/Users/hanskrio/opt/miniconda3/envs/boptestgym/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/hanskrio/opt/miniconda3/envs/boptestgym/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Using mps device
